In [1]:
import pandas as pd
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings
from parse_custom_report import *
from yandex_disk_func import *
from db_func import *
from one_target_parse_func import *

pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()


# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
report_video_dict = {}
report_banner_dict = {}
report_video_views_dict = {}

In [4]:
# вызываем функцию, которая проходит по файлам в определенной папке на Яндекс диске
# если в файл эксель, то она его парсит
# в зависимости от типа креатива (Видео, Баннер или агрегированные просмотры)
# статистика сохраняется в соответсвующий словарь
get_data_from_ya_folder(yandex_folders, main_folder, file_path, yandex_token,
                                              report_video_dict, report_banner_dict, report_video_views_dict, flag='new')

/01_new_sources_files
astralab_banner
gnezdo_banner
/02_regular_reports


In [5]:
# report_video_dict

In [6]:
# report_banner_dict

In [7]:
# report_video_views_dict

In [8]:
# создаем общий список названий полей и типов данных
# он подходит и для video, и для banner
# специфические поля для video Отдельно описали в функции при создании таблицы в БД
# этот список передаем в БД для создания новой таблицы

db_vars_str = [
            'date nvarchar(10)',
            'source nvarchar(20)',
            'format_type nvarchar(20)',
            'report_type nvarchar(20)',
            'product nvarchar(150)',
            'impressions int',
            'clicks int',
            'reach float',
            'budget float']

In [9]:
# создаем пустую таблицы в БД
# Статистика по Видео рекламе
reCreateDBTable(server_local, database_local, 'video_stat_table', db_vars_str, table_type='video')

# статистика по Баннерам
reCreateDBTable(server_local, database_local, 'banner_stat_table', db_vars_str, table_type='banner')

Пустая таблица video_stat_table успешно создана в БД custom_tables_NL_2208
Пустая таблица banner_stat_table успешно создана в БД custom_tables_NL_2208


In [10]:
# если есть файлы со статистикой по Видео объявленияем, то запускаем блок обработки этой статистики
if report_video_dict:
    
    # создаем спец.датаФрейм для расчета досмотрров по дням для источника One Target
    one_target_df = pd.DataFrame()
    one_target_views_df = pd.DataFrame()
    
    # перебираем через цикл словарь со статистикой видео объявлений
    # если нам встречаются ключи, которые относятся к one target
    # сохраняем данные в датаФрейм
    # удаляем эти ключи в словаре
    for key in list(report_video_dict.keys()):
        if 'one_target' in key:
            one_target_df = pd.concat([one_target_df, report_video_dict[key]], ignore_index=True)
            del report_video_dict[key]

    # перебираем через цикл словарь со статистикой видео объявлений
    # если нам встречаются ключи, которые относятся к one target
    # сохраняем данные в датаФрейм
    # удаляем эти ключи в словаре
    for key in list(report_video_views_dict.keys()):
        if 'one_target' in key:
            one_target_views_df = pd.concat([one_target_views_df, report_video_views_dict[key]], ignore_index=True)
            del report_video_views_dict[key]

    # если датаФрейм НЕ пустой со статистикой по источнику one target
    # то вызываем функцию для расчета просмотров по дням
    if not one_target_df.empty:
        report_video_dict['one_target'], report_video_views_dict['one_target'] = get_first_one_target_data(one_target_df, one_target_views_df)

    union_video_df = pd.concat(report_video_dict, ignore_index=True)
    union_video_df = union_video_df.fillna(0)
    
    if 'budget' not in union_video_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
        union_video_df['budget'] = 0
    # создаем общую таблицу со статистикой по video
    # вызываем функцию нормализации данных и прямо в нее передает функцию с объединением датаФрейма
    union_video_df = normalizeDataTypes(union_video_df, table_type='video')
    
    # приводим название полей к итоговому стандарту
    union_video_df = union_video_df.rename(columns={'25': 'video_views_25', '50': 'video_views_50',
                                                   '75': 'video_views_75', '100': 'video_views_100'})
    
    # записываем статистику в БД
    downloadTableToDB(server_local, database_local, 'video_stat_table', union_video_df)

NameError: name 'union_video_df' is not defined

In [12]:
# если есть статистика по Баннерам, то запускаем блок обработки таких данных
if report_banner_dict:

    # создаем один большой датаФрейм со статистикой по баннерам
    union_banner_df = pd.concat(report_banner_dict, ignore_index=True)
    if 'views' not in union_banner_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
        union_banner_df['views'] = 0
    if 'reach' not in union_banner_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
        union_banner_df['reach'] = 0

    if 'budget' not in union_banner_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
        union_banner_df['budget'] = 0
        
    union_banner_df = union_banner_df.fillna(0)
    # нормализуем типы данных 
    union_banner_df = normalizeDataTypes(union_banner_df, table_type='banner') 
    
    # записываем статистику в БД
    downloadTableToDB(server_local, database_local, 'banner_stat_table', union_banner_df)

Connect success
Скрипт запущен 2024-09-25 16:14:01.403147
Данные добавлены в БД: custom_tables_NL_2208, таблица: banner_stat_table
Скрипт отработал 2024-09-25 16:14:01.425140
Время выполнения задачи: 0:00:00.021993
Загрузка завершена. Данные успешно добавлены в БД: custom_tables_NL_2208
##################################################



In [13]:
db_vars_str = [
            'start_period nvarchar(10)',
            'end_period nvarchar(10)',
            'min_date nvarchar(10)',
            'max_date nvarchar(10)',
            'source nvarchar(20)',
            'product nvarchar(100)',
            'report_type nvarchar(20)']

In [14]:
# создаем пустую таблицы в БД
# Статистика по Видео рекламе
reCreateDBTable(server_local, database_local, 'one_target_video_views', db_vars_str, table_type='video')

Пустая таблица one_target_video_views успешно создана в БД custom_tables_NL_2208


In [15]:
# если есть статистика по досмотрам, то нормализуем данные и заливаем в БД
if report_video_views_dict:
    # вызываем функцию нормализации данных для таблицы досмотров
    report_video_views_df = normalizeDataTypes(pd.concat(report_video_views_dict, ignore_index=True), 
                                                                                table_type='video_views') 
    
    report_video_views_df =  report_video_views_df.rename(columns={
        '25': 'video_views_25', '50': 'video_views_50', '75': 'video_views_75', '100': 'video_views_100'
    })

    # вызываем функцию для добавления в БД таблицы с досмотрами
    downloadTableToDB(server_local, database_local, 'one_target_video_views', report_video_views_df)

In [ ]:
%reset -f